In [135]:
# 필요한 모듈 불러오기 
import requests
import scrapy
import re
from scrapy.http import TextResponse

In [136]:
# 스크래피 프로젝트 생성
!rm -rf my_area
!scrapy startproject my_area

New Scrapy project 'my_area', using template directory '/home/ubuntu/.pyenv/versions/3.6.9/envs/python3/lib/python3.6/site-packages/scrapy/templates/project', created in:
    /home/ubuntu/python3/notebook/my_area

You can start your first spider with:
    cd my_area
    scrapy genspider example example.com


In [137]:
# items.py 설정하여 수집할 데이터 설정
#    good = scrapy.Field()
    

In [138]:
%%writefile my_area/my_area/items.py

import scrapy

class My_areaItem(scrapy.Item):
    title = scrapy.Field()
    comment = scrapy.Field()
    day = scrapy.Field()
    link = scrapy.Field()

Overwriting my_area/my_area/items.py


In [139]:
# url 설정하여 들어갈 링크지정 / best contects 5개
url = "https://m.naver.com/history/?menu=PLACE_09"
req = requests.get(url)
response = TextResponse(req.url, body=req.text, encoding="utf-8")
links = response.xpath('//*[@id="mflick"]/div/div[2]/div/div[1]/div[1]')[0].xpath('//*[@class="cvr_list_item"]/a/@href').extract()
print(links[0])
len(links)

https://m.blog.naver.com/hot2hot2seoul/222049999104


5

In [190]:
# link 수집
link = response.urljoin(links[])
link

'https://m.blog.naver.com/mirae_saram/222044126140'

In [191]:
req = requests.get(link)
response = TextResponse(req.url, body=req.text, encoding="utf-8")

In [189]:
links[0]

'https://m.blog.naver.com/hot2hot2seoul/222049999104'

In [192]:
response

<200 https://m.blog.naver.com/mirae_saram/222044126140>

In [193]:
title = response.xpath('//*[@id="SE-5bdbe95a-a7cb-4073-ac6d-d02355f5cc01"]/text()').extract()[0]
title

'[사람나래] 자전거 타고 떠나는 한강'

In [150]:
# 데이터 확인 구문
title = response.xpath('//*[@id="SE-34941879-7272-46c4-b494-066d46aaec3b"]/text()').extract()[0]
title

'낮에는 등산 맛집, 밤에는 야경 맛집! 매봉산 등산로&팔각정'

In [151]:
title = response.xpath('//*[@id="SE-34941879-7272-46c4-b494-066d46aaec3b"]/text()').extract()[0]
day = response.xpath('//*[@id="SE-0bcb5c34-10d6-46a2-976d-0437c9a0bf58"]/div/div/div[3]/p/text()').extract()[0].strip()
comment = response.xpath('//*[@id="ct"]/div[4]/div[3]/div/div[2]/a[1]/em/text()').extract()[0]


title, day, comment, link

('낮에는 등산 맛집, 밤에는 야경 맛집! 매봉산 등산로&팔각정',
 '2020. 8. 3. 16:40',
 '2',
 'https://m.blog.naver.com/hot2hot2seoul/222049999104')

In [117]:
# spider.py 작성

In [ ]:
# 날짜는 같음

//*[@id="SE-0bcb5c34-10d6-46a2-976d-0437c9a0bf58"]/div/div/div[3]/p
//*[@id="SE-0bcb5c34-10d6-46a2-976d-0437c9a0bf58"]/div/div/div[3]/p
# 제목끼리 xpath가 다름

//*[@id="SE-5bdbe95a-a7cb-4073-ac6d-d02355f5cc01"]
//*[@id="SE-34941879-7272-46c4-b494-066d46aaec3b"]

In [161]:
%%writefile my_area/my_area/spiders/spider.py
import scrapy
from my_area.items import My_areaItem

class AreaSpider(scrapy.Spider):
    name = "MyArea"
    allow_domain = ["https://m.naver.com"]
    start_urls = ["https://m.naver.com/history/?menu=PLACE_09"]
    
    def parse(self, response):
        links = response.xpath('//*[@id="mflick"]/div/div[2]/div/div[1]/div[1]')[0].xpath('//*[@class="cvr_list_item"]/a/@href').extract()
        for link in links:
            link = response.urljoin(link)
            yield scrapy.Request(link, callback=self.parse_contents)
                        
    def parse_contents(self, response):
        item = My_areaItem()
        item["title"] = response.xpath('//*[@id="SE-34941879-7272-46c4-b494-066d46aaec3b"]/text()').extract()[0]
        item["day"] = response.xpath('//*[@id="SE-0bcb5c34-10d6-46a2-976d-0437c9a0bf58"]/div/div/div[3]/p/text()').extract()[0].strip()
        item["comment"] = response.xpath('//*[@id="ct"]/div[4]/div[3]/div/div[2]/a[1]/em/text()').extract()[0]
        item["link"] = response.url
        yield item

Overwriting my_area/my_area/spiders/spider.py


In [174]:
%%writefile run.sh
rm my_area/myarea.csv
cd my_area
scrapy crawl MyArea -o myarea.csv

Overwriting run.sh


In [175]:
!chmod +x run.sh

In [176]:
!./run.sh

2020-08-17 17:44:17 [scrapy.utils.log] INFO: Scrapy 2.3.0 started (bot: my_area)
2020-08-17 17:44:17 [scrapy.utils.log] INFO: Versions: lxml 4.5.2.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.6.9 (default, Jul 10 2020, 02:27:22) - [GCC 7.5.0], pyOpenSSL 19.1.0 (OpenSSL 1.1.1g  21 Apr 2020), cryptography 3.0, Platform Linux-5.3.0-1032-aws-x86_64-with-debian-buster-sid
2020-08-17 17:44:17 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.epollreactor.EPollReactor
2020-08-17 17:44:17 [scrapy.crawler] INFO: Overridden settings:
{'BOT_NAME': 'my_area',
 'NEWSPIDER_MODULE': 'my_area.spiders',
 'SPIDER_MODULES': ['my_area.spiders']}
2020-08-17 17:44:17 [scrapy.extensions.telnet] INFO: Telnet Password: 18c04c2dc0353776
2020-08-17 17:44:17 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.feedexport.Fe

2020-08-17 17:44:18 [scrapy.core.scraper] ERROR: Spider error processing <GET https://m.blog.naver.com/mcstkorea/222008543347> (referer: https://m.naver.com/history/?menu=PLACE_09)
Traceback (most recent call last):
  File "/home/ubuntu/.pyenv/versions/3.6.9/envs/python3/lib/python3.6/site-packages/scrapy/utils/defer.py", line 120, in iter_errback
    yield next(it)
  File "/home/ubuntu/.pyenv/versions/3.6.9/envs/python3/lib/python3.6/site-packages/scrapy/utils/python.py", line 347, in __next__
    return next(self.data)
  File "/home/ubuntu/.pyenv/versions/3.6.9/envs/python3/lib/python3.6/site-packages/scrapy/utils/python.py", line 347, in __next__
    return next(self.data)
  File "/home/ubuntu/.pyenv/versions/3.6.9/envs/python3/lib/python3.6/site-packages/scrapy/core/spidermw.py", line 64, in _evaluate_iterable
    for r in iterable:
  File "/home/ubuntu/.pyenv/versions/3.6.9/envs/python3/lib/python3.6/site-packages/scrapy/spidermiddlewares/offsite.py", line 29, in process_spider_ou

In [177]:
import pandas as pd

df = pd.read_csv("my_area/myarea.csv")
df.tail(5)

,comment,day,link,title
0,2,2020. 8. 3. 16:40,https://m.blog.naver.com/hot2hot2seoul/2220499...,"낮에는 등산 맛집, 밤에는 야경 맛집! 매봉산 등산로&팔각정"


In [67]:
# 블로그로 들어가서 좋아요 갯수 크롤링
good_link = links[1].split('/')
good_link[3], good_link[4]

('mirae_saram', '222044126140')

In [68]:
url = "https://blog.like.naver.com/v1/search/contents?suppress_response_codes=\
        true&q=BLOG[{}_{}]&isDuplication=true".format(good_link[3], good_link[4])
response = requests.get(url)

In [69]:
ab = re.findall("\"count\":([\d]+)", response.text)[0] 
ab

'25'